In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
from time import time

# डेटाबेस इंजन बनाएं
engine = create_engine('sqlite:///inventory.db')

# डेटा डायरेक्टरी में सभी फाइलों के लिए लूप चलाएं
for file in os.listdir('data'):
    if file.endswith('.csv'):
        
        t_start = time()
        table_name = file[:-4]
        
        print(f"Processing {file}...")

        # चंक्स में फ़ाइल पढ़ने के लिए इटरेटर बनाएं
        df_iterator = pd.read_csv('data/' + file, iterator=True, chunksize=100000)

        # पहला चंक पढ़ें
        df = next(df_iterator)

        # पहले चंक से कॉलम की संख्या और रॉ की शुरुआती संख्या ले लें
        total_rows = len(df)
        num_cols = df.shape[1]

        # पहला चंक टेबल को 'replace' मोड में लिखता है
        df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
        
        # बाकी बचे हुए चंक्स को लूप में चलाकर 'append' करें और रॉ की गिनती करें
        for chunk in df_iterator:
            chunk.to_sql(name=table_name, con=engine, if_exists='append', index=False)
            # कुल रॉ में इस चंक की रॉ जोड़ें
            total_rows += len(chunk)
        
        t_end = time()
        print(f"Completed ingesting {file} in {t_end - t_start:.3f} seconds.")
        # आखिर में कुल shape प्रिंट करें
        print(f"  - Total Shape: ({total_rows}, {num_cols})\n")

Processing begin_inventory.csv...
Completed ingesting begin_inventory.csv in 3.097 seconds.
  - Total Shape: (206529, 9)

Processing end_inventory.csv...
Completed ingesting end_inventory.csv in 3.452 seconds.
  - Total Shape: (224489, 9)

Processing purchases.csv...
Completed ingesting purchases.csv in 59.889 seconds.
  - Total Shape: (2372474, 16)

Processing purchase_prices.csv...
Completed ingesting purchase_prices.csv in 0.405 seconds.
  - Total Shape: (12261, 9)

Processing sales.csv...
Completed ingesting sales.csv in 266.342 seconds.
  - Total Shape: (12825363, 14)

Processing vendor_invoice.csv...
Completed ingesting vendor_invoice.csv in 0.287 seconds.
  - Total Shape: (5543, 10)

